In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import sqlite3

from functools import reduce

In [2]:
cols = ['code', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

# Importing temperature, precipitation, PMDI data
tmp = pd.read_fwf('../data/raw/climdiv-tmpcst-v1.0.0-20190705', header=None, names=cols, 
                  dtype={0: str}, na_values=-99.90)

pcp = pd.read_fwf('../data/raw/climdiv-pcpnst-v1.0.0-20190705', header=None, names=cols, 
                  dtype={0: str}, na_values=-9.99) 

pmdi = pd.read_fwf('../data/raw/climdiv-pmdist-v1.0.0-20190705', header=None, names=cols, 
                   dtype={0: str}, na_values=-99.99)

In [3]:
def extract_state(code: str):
    return code[0:3]

def extract_year(code: str):
    return code[6:11]


pcp['state'] = pcp.code.apply(extract_state)
pcp['year'] = pcp.code.apply(extract_year).astype(int)

tmp['state'] = tmp.code.apply(extract_state)
tmp['year'] = tmp.code.apply(extract_year).astype(int)

pmdi['state'] = pmdi.code.apply(extract_state)
pmdi['year'] = pmdi.code.apply(extract_year).astype(int)

pcp = pcp.drop('code', axis=1)
tmp = tmp.drop('code', axis=1)
pmdi = pmdi.drop('code', axis=1)

# Rearrange columns
cols = tmp.columns.tolist()
cols = cols[-2:] + cols[:-2]
tmp = tmp[cols]
pcp = pcp[cols]
pmdi = pmdi[cols]

In [4]:
# Now that state code is extracted, convert to int for easier filtering
tmp['state'] = tmp.state.astype(int)
pcp['state'] = pcp.state.astype(int)
pmdi['state'] = pmdi.state.astype(int)

# Drop region and national rows
tmp = tmp[tmp.state <= 50]
pcp = pcp[pcp.state <= 50]
pmdi = pmdi[pmdi.state <= 50]

In [5]:
state_dict = {1: 'Alabama',
              2: 'Arizona',
              3: 'Arkansas',
              4: 'California',
              5: 'Colorado',
              6: 'Connecticut',
              7: 'Delaware',
              8: 'Florida',
              9: 'Georgia',
              10: 'Idaho',
              11: 'Illinois',
              12: 'Indiana',
              13: 'Iowa',
              14: 'Kansas',
              15: 'Kentucky',
              16: 'Louisiana',
              17: 'Maine',
              18: 'Maryland',
              19: 'Massachusetts',
              20: 'Michigan',
              21: 'Minnesota',
              22: 'Mississippi',
              23: 'Missouri',
              24: 'Montana',
              25: 'Nebraska',
              26: 'Nevada',
              27: 'New Hampshire',
              28: 'New Jersey',
              29: 'New Mexico',
              30: 'New York',
              31: 'North Carolina',
              32: 'North Dakota',
              33: 'Ohio',
              34: 'Oklahoma',
              35: 'Oregon',
              36: 'Pennsylvania',
              37: 'Rhode Island', 
              38: 'South Carolina',
              39: 'South Dakota',
              40: 'Tennessee',
              41: 'Texas',
              42: 'Utah',
              43: 'Vermont', 
              44: 'Virginia',
              45: 'Washington',
              46: 'West Virginia',
              47: 'Wisconsin',
              48: 'Wyoming',
              50: 'Alaska'}

tmp['state'] = tmp.state.map(state_dict)
pcp['state'] = pcp.state.map(state_dict)
pmdi['state'] = pmdi.state.map(state_dict)

In [6]:
# Melt DataFrames
tmp = tmp.melt(id_vars=['state', 'year'], var_name='month', value_name='avg_temp', 
               value_vars=[i for i in range(1, 13)]) \
         .sort_values(['state', 'year', 'month']) \
         .reset_index(drop=True)
tmp['month'] = tmp.month.astype(int)

pcp = pcp.melt(id_vars=['state', 'year'], var_name='month', value_name='precipitation',
               value_vars=[i for i in range(1, 13)]) \
         .sort_values(['state', 'year', 'month']) \
         .reset_index(drop=True)
pcp['month'] = pcp.month.astype(int)

pmdi = pmdi.melt(id_vars=['state', 'year'], var_name='month', value_name='pmdi',
                 value_vars=[i for i in range(1, 13)]) \
           .sort_values(['state', 'year', 'month']) \
           .reset_index(drop=True)
pmdi['month'] = pmdi.month.astype(int)

In [7]:
# Merge multiple DataFrames
dfs = [tmp, pcp, pmdi]

met = reduce(lambda left, right: pd.merge(left, right, on=['state', 'year', 'month']), dfs)
met.head()

,state,year,month,avg_temp,precipitation,pmdi
0,Alabama,1895,1,43.1,7.52,0.78
1,Alabama,1895,2,37.4,2.66,-0.59
2,Alabama,1895,3,54.5,7.62,0.46
3,Alabama,1895,4,63.4,3.58,-0.54
4,Alabama,1895,5,69.5,3.78,-0.55


In [8]:
conn = sqlite3.connect('../data/raw/storms.db')
c = conn.cursor()

query = '''
          SELECT BEGIN_YEARMONTH AS YEARMONTH,
                 STATE,
                 COUNT(*) AS EVENT_COUNT
            FROM details 
        GROUP BY YEARMONTH,
                 STATE
        ORDER BY STATE,
                 YEARMONTH;
        '''

'''
Possible columns of interest: 

EVENT_TYPE
SUM(INJURIES_DIRECT) AS INJURIES_DIRECT,
SUM(INJURIES_INDIRECT) AS INJURIES_INDIRECT,
SUM(DEATHS_DIRECT) AS DEATHS_DIRECT,
SUM(DEATHS_INDIRECT) AS DEATHS_INDIRECT
DAMAGE_PROPERTY,
DAMAGE_CROPS,
MAGNITUDE,
MAGNITUDE_TYPE,
FLOOD_CAUSE,
TOR_F_SCALE,
TOR_LENGTH,
TOR_WIDTH
'''

results = c.execute(query)
df = pd.DataFrame(results.fetchall())

# Set column names to df in lowercase
names = [description[0].lower() for description in results.description]
df.columns = names

conn.close()

In [9]:
def separate(yearmonth: int):
    yearmonth = str(yearmonth)
    return int(yearmonth[:4]), int(yearmonth[-2:])


df[['year', 'month']] = pd.DataFrame(df.yearmonth.apply(separate).tolist(), index=df.index)
df = df.drop('yearmonth', axis=1)
df['year'] = df.year.astype(int)
df['month'] = df.month.astype(int)

cols = df.columns.tolist()
cols = cols[-2:] + cols[:-2]
df = df[cols]

In [10]:
# Capitalize only first letter of state and drop non-state rows
df['state'] = df.state.str.capitalize()
df = df[df['state'].isin(state_dict.values())].reset_index(drop=True)

In [11]:
df = pd.merge(df, met, on=['year', 'month', 'state']) \
       .sort_values(['state', 'year', 'month']) \
       .reset_index(drop=True)

In [12]:
df.head()

,year,month,state,event_count,avg_temp,precipitation,pmdi
0,1950,4,Alabama,2,59.2,3.48,-1.42
1,1951,2,Alabama,1,49.3,3.39,-1.41
2,1951,6,Alabama,1,78.4,4.68,-0.80
3,1951,11,Alabama,3,48.7,3.68,-1.60
4,1952,1,Alabama,1,52.4,3.96,-1.12


In [13]:
df.describe()

,year,month,event_count,avg_temp,precipitation,pmdi
count,21573.000000,21573.000000,21573.000000,21573.000000,21573.000000,21573.000000
mean,1990.991517,6.449451,58.414546,56.879980,3.335779,1.463024
std,19.080810,3.103357,96.391015,16.323095,2.000699,11.374322
min,1950.000000,1.000000,1.000000,-4.700000,0.000000,-9.090000
25%,1975.000000,4.000000,5.000000,45.600000,1.840000,-1.500000
50%,1996.000000,6.000000,21.000000,59.200000,3.090000,0.380000
75%,2007.000000,9.000000,72.000000,69.900000,4.500000,2.050000
max,2019.000000,12.000000,1596.000000,89.200000,14.450000,99.990000
